## インストール

In [6]:
pip install pingouin
pip install plotnine
pip install mysql-connector-python
pip install numexpr
pip install python-dotenv
pip install japanize-matplotlib


SyntaxError: invalid syntax (<ipython-input-6-5ad1bb845e3b>, line 2)

In [1]:
from itertools import combinations

import csv
import MySQLdb
import mysql.connector as mydb
import pandas as pd
import pandas.io.sql as psql
import numpy as np
from pingouin import kruskal, mwu, multicomp
from plotnine import *


In [5]:
%load_ext dotenv
%dotenv


In [7]:
host = os.environ.get('HOST')
user = os.environ.get('USER_NAME')
password = os.environ.get('PASSWORD')
database = os.environ.get('DATABASE')
DB_CONFIG = dict(host=host, user=user, password=password, database=database)

def check_db_connection(db_config=DB_CONFIG):
  connection = mydb.connect(**db_config)
  connection.ping(reconnect=True)
  # 接続できているかどうか確認
  print(connection.is_connected())


check_db_connection(DB_CONFIG)


True


In [ ]:
UI_CONDITION = ("question", "perspective", "non")
THEME_CONDITION = ("tpp", "airPollution", "vietnamTrip")

In [11]:

def get_dataframe_from_db(sql):
  """
  db上のデータをデータフレームとして取得
  """
  conn = mydb.connect(**DB_CONFIG)
  df_read = pd.read_sql(sql, conn)
  conn.close()
  return df_read


def convert_csv_to_array(csv_path):
  """
  csvファイルの中身を一次元配列へ変換
  """
  result_arr = []
  with open(csv_path) as f:
      reader = csv.reader(f)
      for row in reader:
        result_arr.append(str(row[0]))
  return result_arr


def convert_singleArray_to_dataframe(array, columnName):
  df = pd.DataFrame(array, columns=[columnName],)
  return df


## 分析対象とする実験協力者

- ウェブ検索タスクにて意見文章の提出まで行い，事後アンケートへの回答が完了しているワーカー
- タスク所要時間が外れ値ではないワーカー

In [27]:

# ウェブ検索タスクと事後アンケートへの回答の両方が完了しているワーカー
questionnaire_answered_crowdIds_csv_path = './data/questionnaire_answered_crowdId.csv'
questionnaire_answered_crowdIds = convert_csv_to_array(questionnaire_answered_crowdIds_csv_path)
questionnaire_answered_crowdIds_df = convert_singleArray_to_dataframe(
    questionnaire_answered_crowdIds, 'crowdId')
users_who_complete_webSearchTask_df = get_dataframe_from_db("select * from users where hasFinishedWebSearchTask")
users_who_completed_all_task_df =  users_who_complete_webSearchTask_df[users_who_complete_webSearchTask_df['crowdId'].isin(questionnaire_answered_crowdIds)]

all_task_done_user_crowdIds = users_who_completed_all_task_df['crowdId'].tolist()
# タスク所要時間の外れ値処理


def get_dataframe_without_outlier(df, target_column):
    q1 = df[target_column].quantile(
        0.25)
    q3 = df[target_column].quantile(
        0.75)
    iqr = q3 - q1
    threshold_upper = q3 + 1.5 * iqr
    threshold_lower = q1 - 1.5 * iqr
    df["threshold_lower"] = threshold_lower
    df["threshold_upper"] = threshold_upper

    df_without_outlier = df[(df[target_column] < threshold_upper) & (
        df[target_column] > threshold_lower)]
    return df_without_outlier

task_time_df = get_dataframe_from_db("select * from taskTime;")
task_time_with_users_done_all_task_df = task_time_df[task_time_df['crowdId'].isin(
    all_task_done_user_crowdIds)]
task_time_with_users_done_all_task_df["taskTimeDuration"] = (
    task_time_with_users_done_all_task_df["finished_at"]-task_time_with_users_done_all_task_df["reStarted_at"]).map(lambda x: x.total_seconds())

valid_users_df = get_dataframe_without_outlier(
    task_time_with_users_done_all_task_df, 'taskTimeDuration')
print(len(valid_users_df)) # 264人





264


<ipython-input-11-751477a6ed0c>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
<ipython-input-11-751477a6ed0c>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
<ipython-input-27-225ee121da04>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<ipython-input-27-225ee121da04>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 